In [1]:
import numpy as np
import math

from astropy.io import fits
from astropy.units import degree, kpc
from astropy.coordinates import SkyCoord

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = fits.open('delta_transmission_RMplate.fits')
len(data)

1247

### Data Model

**PrimaryHDU**

In [4]:
data[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 

**BinaryTableHDU**

In [5]:
data[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional binary table                     
NAXIS1  =                   24 / width of table in bytes                        
NAXIS2  =                 3466 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group (required keyword)              
TFIELDS =                    3 / number of fields in each row                   
TTYPE1  = 'LAMBDA  '           / label for field   1                            
TFORM1  = 'D       '           / data format of field: 8-byte DOUBLE            
TUNIT1  = 'Angstrom'           / physical unit of field                         
TTYPE2  = 'DELTA_T '           / label for field   2                            
TFORM2  = 'D       '        

In [6]:
data[1].data.shape

(3466,)

In [7]:
data[1].data['LAMBDA'][:10]

array([ 4506.6,  4506.8,  4507. ,  4507.2,  4507.4,  4507.6,  4507.8,
        4508. ,  4508.2,  4508.4])

In [8]:
data[1].data['DELTA_T'][:10]

array([ 0.32041982,  0.32820452,  0.32227549,  0.30720392,  0.32257604,
        0.37425373,  0.41448467,  0.43779907,  0.44572165,  0.42081868])

In [9]:
data[1].data['RCOMOV'][:10]

array([ 4181.91557086,  4182.03612559,  4182.15668033,  4182.27723506,
        4182.3977898 ,  4182.51834453,  4182.63887691,  4182.75938488,
        4182.87989285,  4183.00040082])

## Getting Coordinates

In [10]:
ra_mean = 213.704
dec_mean = 53.083

### Vikrant's codes

In [11]:
x_list = []
y_list = []
z_list = []
delta_list = []

In [55]:
nskewers = 100
ixs = np.random.choice(np.arange(1, len(data)), 1000, replace=False)

result = []
for i in ixs:
    ra = (data[i].header['RA'] - ra_mean) * np.pi / 360
    dec = (data[i].header['DEC'] - dec_mean) * np.pi / 360
    rcomov = data[i].data['RCOMOV']
    value = data[i].data['DELTA_T']
    
    x = np.sin(dec) * np.cos(ra) * rcomov
    y = np.sin(dec) * np.sin(ra) * rcomov
    z = np.cos(dec) * rcomov
    
    x_list.append(x)
    y_list.append(y)
    z_list.append(z)
    delta_list.append(value)

In [56]:
# coordinates
x = np.hstack(x_list)
y = np.hstack(y_list)
z = np.hstack(z_list)
delta = np.hstack(delta_list)

In [57]:
# shifted cordinates
x -= x.min() - 5
y -= y.min() - 5
z -= z.min() - 5

In [58]:
# select a subsample
ind = np.where(z < 500)[0]
x = x[ind]
y = y[ind]
z = z[ind]
delta = delta[ind]

In [34]:
from plotly.offline import plot, iplot
import plotly.graph_objs as go

In [35]:
trace = go.Scatter3d(x=x, y=y, z=z, mode='markers')
data_3d = [trace]

fig = go.Figure(data=data_3d)
plot(fig)

'file:///Users/yulong/Codes/astro-void/interpolation/temp-plot.html'

In [89]:
noise = 0.1 * np.ones(len(z))

p = np.vstack([x.T, y.T, z.T, noise.T, delta.T]).T
p.tofile('pixel_data.bin')
# binaryData = np.vstack(result)

In [59]:
npx_x = np.floor((x.max() - x.min()) / 4.)
npx_y = np.floor((y.max() - y.min()) / 4.)
npx_z = np.floor((z.max() - z.min()) / 4.)

In [60]:
npx_x

27.0

In [61]:
npx_y

0.0

In [62]:
y.max()

6.1769532567622836

In [63]:
y.min()

5.0

In [64]:
(y.max() - y.min()) / 4

0.29423831419057089

In [91]:
# write config file
cf = open("void.cfg", "w")
cf.write("lx = %f\n" % (x.max() + 5))
cf.write("ly = %f\n" % (y.max() + 5))
cf.write("lz = %f\n" % (z.max() + 5))
cf.write("num_pixels = %i\n" % len(x))
cf.write("map_nx = %i\n" % npx_x)
cf.write("map_ny = %i\n" % npx_y)
cf.write("map_nz = %i\n" % npx_z)
cf.write("corr_var_s = 0.05\n")
cf.write("corr_l_perp = 2.5\n")
cf.write("corr_l_para = 2.5\n")
cf.write("pcg_tol = 1.0e-5\n")
cf.write("pcg_max_iter = 1000\n")
cf.close()

In [97]:
import subprocess

In [99]:
subprocess.run(['./dachshund.exe', 'void.cfg'], stdout=subprocess.PIPE)

KeyboardInterrupt: 

In [96]:
! ./dachshund.exe void.cfg

Reading config file void.cfg.

Running dachshund with:
  pixels/map:
    lx ly lz = 105.994766 11.012837 104.995138
    npix = 19314
    map nx ny nz = 22 0 23, n = 0
  signal:
    corr var_s = 0.050000
    l_perp = 2.500000    l_para = 2.500000
  WF x solve: 1
  PCG:
    max iter = 1000    step r = 50
    tol = 0.000010
  options:
    map covar = 0
  paths:
    pixel data = pixel_data.bin
    map = map.bin

Reading pixel data.
Preparing for x solve.
Starting solve.
[PCG] Solving 19314 x 19314 problem.
    Goal |r| < (tol) |b| = 5.122109e-03,  delta = 4.372667e+04
    iter 1, |r| 1.283431e+03, delta 2.745327e+05
    iter 2, |r| 1.638035e+03, delta 4.471929e+05
^C
